<a href="https://colab.research.google.com/github/konschak/WebscrappingAP/blob/main/WebScrappingAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install requests
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=731d549e15710439d4cac4e47b8077642881c75d9f623648cc99879b9b7a9a27
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4



Tutorial
"https://www.youtube.com/watch?v=VGroXCEaBiA&list=LL"
Código
"https://github.com/fabrimatt/webscrapping/blob/main/scrapping.py"

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import math

In [11]:
url = 'https://www.chavesnamao.com.br/apartamentos-a-venda/pr-curitiba/?utm_source=google&utm_medium=conversao_venda&utm_campaign=conversao_venda_pr_cwb&utm_content=&gclid=Cj0KCQjw5f2lBhCkARIsAHeTvlgjJp5ARf3TRrjw_2-ALBhu4JcJo0MeJibemBCcMosjALMJrUBzFUkaArTHEALw_wcB'

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

site = requests.get(url, headers=headers)

In [12]:
soup = BeautifulSoup(site.content, 'html.parser')

In [13]:
qtd_itens = soup.find('b').get_text().strip()
print(qtd_itens)

32.557


In [14]:
qtd_itens = qtd_itens.replace('.','')
qtd_itens

'32557'

In [15]:
ultima_pagina = math.ceil(int(qtd_itens)/ 10)
ultima_pagina

3256

In [57]:
dic_produtos = {'preco':[], 'metros_quadrados':[], 'num_quartos':[], 'num_banheiro':[], 'num_garagem':[], 'endereco':[], 'link' : []}

In [52]:
soup = BeautifulSoup(site.content, 'html.parser')
soup.select_one('ul', class_=re.compile('list im'))

<ul class="grid"><li></li><li></li><li></li><li></li><li></li><li></li><li></li><li></li><li></li></ul>

In [63]:
for i in range(1, ultima_pagina):
    url_pag = f'https://www.chavesnamao.com.br/apartamentos-a-venda/pr-curitiba/?pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    produtos = soup.find_all('div', class_=re.compile('imoveis__Card'))

    for produto in produtos:
        preco = produto.find('p', class_=re.compile('price')).contents[0].get_text().strip()
        metros_quadrados = produto.find('ul', class_=re.compile('list im')).contents[0].get_text().strip().replace('m²','')
        try:
          num_quartos = produto.find('ul', class_=re.compile('list im')).contents[1].get_text().strip().replace(' Quartos','').replace(' Quarto','')
        except IndexError:
          num_quartos = 0
        try:
          num_banheiro = produto.find('ul', class_=re.compile('list im')).contents[2].get_text().strip().replace('Banheiros','').replace('Banheiro','')
        except IndexError:
          num_banheiro = 0
        try:
          num_garagem = produto.find('ul', class_=re.compile('list im')).contents[3].get_text().strip().replace('Garagens','').replace('Garagem','')
        except IndexError:
          num_garagem = 0
        endereco = produto.find('address').contents[1].get_text().strip().replace(', Curitiba','')
        link = produto.find('span', class_=re.compile('link')).contents[0].get('href')

        print(preco, metros_quadrados,num_quartos, num_banheiro,endereco)
        dic_produtos['preco'].append(preco)
        dic_produtos['metros_quadrados'].append(metros_quadrados)
        dic_produtos['num_quartos'].append(num_quartos)
        dic_produtos['num_banheiro'].append(num_banheiro)
        dic_produtos['num_garagem'].append(num_garagem)
        dic_produtos['endereco'].append(endereco)
        dic_produtos['link'].append(link)

R$ 339.900 48 a 15 2  1   Capão Raso
R$ 390.000 133 3  1   Portão
R$ 930.000 68 2  2   Vila Izabel
R$ 298.000 64 2  1   Santa Cândida
R$ 890.000 122 2  3   São Francisco
R$ 257.900 19 a 50 2  1   Centro
R$ 320.000 32 1  1   Centro
R$ 180.000 44 2  1   Santa Cândida
R$ 220.000 45 2  1   Cidade Industrial
R$ 2.650.000 351 4  5   Água Verde
R$ 895.500 77 2  2   Água Verde
R$ 251.100 24 1  1   Centro
R$ 680.000 172 3  2   Alto da Glória
R$ 626.026 53 2  2   Água Verde
R$ 855.000 74 a 142 2  2   Campo Comprido
R$ 264.000 28 a 111 1 a 2  1   Portão
R$ 185.000 62 2  1   Jardim Botânico
R$ 840.110 67 2  2   Alto da Glória
R$ 2.690.000 230 3  4   Cabral
R$ 629.000 56 2  0 Água Verde
R$ 219.900 17 1  1   Água Verde
R$ 159.900 26 1  1   Cajuru
R$ 320.000 88 3  2   Campina do Siqueira
R$ 185.000 43 2  1   Fazendinha
R$ 1.100.000 125 3  4   Juvevê
R$ 840.110 67 2  3   Alto da Glória
R$ 650.000 135 3  2   Centro
R$ 780.000 74 2  2   Centro
R$ 235.000 55 3  1   Fazendinha
R$ 284.000 45 a 56 2  1   Pr

In [64]:
df = pd.DataFrame(dic_produtos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4575 entries, 0 to 4574
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   preco             4575 non-null   object
 1   metros_quadrados  4575 non-null   object
 2   num_quartos       4575 non-null   object
 3   num_banheiro      4575 non-null   object
 4   num_garagem       4575 non-null   object
 5   endereco          4575 non-null   object
 6   link              4575 non-null   object
dtypes: object(7)
memory usage: 250.3+ KB


In [65]:
df

,preco,metros_quadrados,num_quartos,num_banheiro,num_garagem,endereco,link
0,R$ 339.900,48 a 15,2,1,1 Garagem,Capão Raso,https://www.chavesnamao.com.br/lancamento/gard...
1,R$ 390.000,133,3,1,1 Garagem,Portão,https://www.chavesnamao.com.br/imovel/apartame...
2,R$ 930.000,68,2,2,1 Garagem,Vila Izabel,https://www.chavesnamao.com.br/imovel/apartame...
3,R$ 298.000,64,2,1,1 Garagem,Santa Cândida,https://www.chavesnamao.com.br/imovel/apartame...
4,R$ 890.000,122,2,3,2 Garagens,São Francisco,https://www.chavesnamao.com.br/imovel/apartame...
...,...,...,...,...,...,...,...
4570,R$ 1.929.000,160,3,5,3,Hugo Lange,https://www.chavesnamao.com.br/imovel/apartame...
4571,R$ 415.800,54,2,1,1,Tingui,https://www.chavesnamao.com.br/imovel/apartame...
4572,R$ 2.250.000,228,4,1,3,Cabral,https://www.chavesnamao.com.br/imovel/apartame...
4573,R$ 767.280,74,3,2,1,Tarumã,https://www.chavesnamao.com.br/imovel/apartame...


In [66]:
from google.colab import drive
drive.mount ('/drive')
df.to_csv('/drive/My Drive/Colab Notebooks/Apartamentos.csv', encoding='utf-8', sep=';')

Mounted at /drive
